In [1]:
!pip install -U gigachain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 703.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.7/403.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.9/220.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4

In [ ]:
import requests
import uuid
import os
import base64
from google.colab import userdata

from langchain.chat_models.gigachat import GigaChat
from langchain.schema import HumanMessage, SystemMessage


In [ ]:
def get_token(auth_token, scope='GIGACHAT_API_PERS'):
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"
    rq_uid = str(uuid.uuid4())

    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth_token}'
    }

    payload = {
        'scope': scope
    }

    try:
        response = requests.request("POST", url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Ошибка: {str(e)}")
        return -1

In [ ]:
def get_credentials(giga_client_id: str, giga_key: str):
    credentials = f"{giga_client_id}:{giga_key}"
    encoded_credentials = base64.b64encode(credentials.encode('utf-8')).decode('utf-8')
    return encoded_credentials


In [ ]:
giga_client_id = userdata.get('GIGA_CHAT_ID')
giga_key = userdata.get('GIGA_CHAT_KEY')

credentials=get_credentials(giga_client_id=giga_client_id, giga_key=giga_key)
response = get_token(credentials)
if response != 1:
  giga_token = response.json()['access_token']
else:
    print("Error")


In [ ]:
model = GigaChat(
    credentials=credentials,
    scope="GIGACHAT_API_PERS",
    model="GigaChat",
    verify_ssl_certs=False,
)

In [ ]:
def delete_comments(code: str):
    flag = False
    processed_code = ""

    for line in code.split('\n'):
        if flag and line.strip().startswith('"""'):
            flag = False
        elif line.strip().endswith('"""'):
            flag = True
        elif not(line.strip().startswith('#') or flag):
            processed_code += line+'\n'
    return processed_code[:-1]

import re

def anonymize_code(code):
    # Множество ключевых слов Python и встроенных функций, которые не нужно заменять
    keywords = {
        "def", "return", "class", "if", "else", "elif", "for", "while", "import",
        "from", "as", "with", "print", "input", "and", "or", "not", "in", "is",
        "True", "False", "None", "break", "continue", "pass", "lambda"
    }

    # Шаблоны для поиска имён переменных, функций и классов
    var_pattern = r'\b([a-zA-Z_][a-zA-Z_0-9]*)\b'
    func_pattern = r'\bdef\s+([a-zA-Z_][a-zA-Z_0-9]*)\b'
    class_pattern = r'\bclass\s+([a-zA-Z_][a-zA-Z_0-9]*)\b'

    # Счётчики для переменных, функций и классов
    var_counter = 1
    func_counter = 1
    class_counter = 1

    # Хранение замен для каждого типа идентификаторов
    var_replacements = {}
    func_replacements = {}
    class_replacements = {}

    # Анонимизация классов
    def replace_class(match):
        nonlocal class_counter
        class_name = match.group(1)
        if class_name not in class_replacements:
            class_replacements[class_name] = f'class{class_counter}'
            class_counter += 1
        return f'class {class_replacements[class_name]}'

    # Анонимизация функций
    def replace_func(match):
        nonlocal func_counter
        func_name = match.group(1)
        if func_name not in func_replacements:
            func_replacements[func_name] = f'func{func_counter}'
            func_counter += 1
        return f'def {func_replacements[func_name]}'

    # Анонимизация переменных, исключая ключевые слова Python
    def replace_var(match):
        nonlocal var_counter
        var_name = match.group(1)
        if var_name not in keywords and var_name not in var_replacements:
            var_replacements[var_name] = f'var{var_counter}'
            var_counter += 1
        return var_replacements.get(var_name, var_name)

    # Выполняем замену для классов, функций и переменных
    anonymized_code = re.sub(class_pattern, replace_class, code)
    anonymized_code = re.sub(func_pattern, replace_func, anonymized_code)
    anonymized_code = re.sub(var_pattern, replace_var, anonymized_code)

    return anonymized_code

# Пример использования:
code = '''
class MyClass:
    def my_function(a, b):
        result = a + b
        return result
#Please help me
x = MyClass()
y = x.my_function(2, 3)
print(y)
'''

code_without_comments = delete_comments(code)
anonymized_code = anonymize_code(code_without_comments)
print(anonymized_code)

In [ ]:
!unzip -q hackaton_files.zip

In [ ]:
import pandas as pd

train_df_solutions = pd.read_excel('for_teams/train/solutions.xlsx')
train_df_tasks = pd.read_excel('for_teams/train/tasks.xlsx')
train_df_tests = pd.read_excel('for_teams/train/tests.xlsx')

In [ ]:
for index, row in train_df_solutions.head(10).iterrows():
  student_code = row['student_solution']
  task_id = row['task_id']
  task = train_df_tasks.loc[train_df_tasks['id'] == task_id]['description'].values[0]
  author_code = train_df_tasks.loc[task_id]['author_solution']
  tests = train_df_tests[train_df_tests['task_id'] == task_id]

  print(
  f"""Задача:
    {task}
    Решение студента:
    {student_code}
    Правильное решение (решение студента должно к нему стремится):
    {author_code}
    Тестовые данные:
    {tests}
    """
  )

In [ ]:
def get_promts(task: str, student_code: str, author_code: str, tests: str):
  student_code_without_comments = delete_comments(student_code)
  #anonymized_student_code = anonymize_code(code_without_comments)

  author_code_without_comments = delete_comments(author_code)
  #anonymized_author_code = anonymize_code(code_without_comments)

  promts = []
  promt = f"""Дана задача, код студента, и тесты (открытые и закрытые). Объясни, какие тесты не проходят и в чём может быть ошибка в коде. Не раскрывай детали закрытых тестов и не предлагай конкретное решение. Опиши логику ошибки в 2-3 предложениях.

Задача:
{task}
Решение студента:
```python
{student_code_without_comments}
```
Предполагаемое правильное решение:
```python
{author_code_without_comments}
```
Входные данные:
{tests}"""
  promts.append(promt)

  promt = """
Дай краткую подсказку, которая поможет студенту самостоятельно найти и исправить ошибку. Сосредоточься на логике задачи, избегая упоминания конкретных кодовых строк или закрытых тестов. Ответь одним предложением, выделив аспект, на который нужно обратить внимание, учитывая тип логической ошибки.
"""
  promts.append(promt)
  promt = """
Дай краткую подсказку, которая поможет студенту самостоятельно найти и исправить ошибку. Сосредоточься на логике задачи, избегая упоминания конкретных строк кода или закрытых тестов. В ответе нельзя использовать код или данные с тестов. Ответь одним предложением, выделяя аспект, на который нужно обратить внимание. Пиши простым, понятным языком. Дай указания мне, как студенту.
Вот примеры формулировок:
Обратите внимание …
Вы забыли… (в случае синтаксической ошибки)
Проверьте…
В данном случае…
"""
  promts.append(promt)
  return promts

In [ ]:
for index, row in train_df_solutions.head(30).iterrows():
  student_code = row['student_solution']
  task_id = row['task_id']
  task = train_df_tasks.loc[train_df_tasks['id'] == task_id]['description'].values[0]
  author_code = train_df_tasks.loc[task_id]['author_solution']
  tests = train_df_tests[train_df_tests['task_id'] == task_id]

  promts = [HumanMessage(content=promt_text) for promt_text in get_promts(task, student_code, author_code, tests)]

  history = [
      SystemMessage(
          content="Ты — профессиональный программист и ментор. Твоя задача — помогать студентам находить ошибки в их коде, направляя их к правильному решению, указывая на конкретные ошибки, допущенные студентом. Ты не раскрываешь прямое решение или закрытые тесты. Учитывай как открытые, так и закрытые тесты."
      )
  ]

  for promt in promts:
      history.append(promt)
      res = model.invoke(history)
      history.append(res)

  print(f"\nОтвет системы: {history[-1].content}")
  author_solution = row['author_comment']
  print(f"\nЭталонный ответ: {author_solution}")
  print("=" * 100)
